In [1]:
# -*- coding: utf-8 -*-
import os

from io import StringIO
from dotenv import load_dotenv

import pandas as pd

from api_functions import TelegramBot
from api_functions import YandexDirect
from api_functions import YandexMessengerBot

### Get env vars

In [2]:
load_dotenv()
bot_token = os.getenv("TELEGRAM_API_TOKEN")
token = os.getenv("YANDEX_TOKEN")
yam_token = os.getenv("YAM_TOKEN")

### Telegram & Yandex Bot Token, chat ID and instances

In [3]:
# chat_id = '126841573' # personal
chat_id = '-4090186402' # Мой канал с алёртами
# chat_id = '-1002145547826' # Чат закупки в телеге
chat_id_ya = '0/0/6b2bfa7e-ed2e-4be9-9a12-b753a68a4a3e'
telegram_bot = TelegramBot(bot_token, chat_id)
ya_bot = YandexMessengerBot(yam_token, chat_id_ya)

## Запросы в Яндекс

### Create Instance of Yandex Exporter

In [5]:
yandex = YandexDirect(token)

In [6]:
logins_actual = ['lost-ark-mrt', 'perfect-world-mrt', 'arche-age-mrt', 
                 'allodsonline-games', 'atomicheart-game', 'battle-teams2', 
                 'warface2016-2017warfacerwyndxg', 'revival-astrum']

### Запрос средств на аккаунтах

In [7]:
df = pd.DataFrame(yandex.accounts_budget(logins_actual))

Request was successful
{'data': {'Accounts': [{'EmailNotification': {'PausedByDayBudget': 'Yes', 'Email': 'd.eliseev@corp.mail.ru', 'MoneyWarningValue': 20}, 'AccountDayBudget': None, 'AgencyName': 'РиалВеб', 'Login': 'lost-ark-mrt', 'Currency': 'RUB', 'Amount': '2855828.547667', 'SmsNotification': {'SmsTimeFrom': '09:00', 'MoneyInSms': 'No', 'SmsTimeTo': '21:00', 'MoneyOutSms': 'No', 'PausedByDayBudgetSms': 'Yes'}, 'AmountAvailableForTransfer': '2834303.55', 'AccountID': 39838670}, {'Amount': '769202.26375', 'Login': 'perfect-world-mrt', 'Currency': 'RUB', 'AmountAvailableForTransfer': '768902.26', 'AccountID': 48709557, 'SmsNotification': {'PausedByDayBudgetSms': 'Yes', 'SmsTimeTo': '21:00', 'MoneyOutSms': 'No', 'MoneyInSms': 'No', 'SmsTimeFrom': '09:00'}, 'EmailNotification': {'MoneyWarningValue': 20, 'PausedByDayBudget': 'Yes', 'Email': 'ivan.sidorenko@corp.mail.ru'}, 'AccountDayBudget': None, 'AgencyName': 'РиалВеб'}, {'AgencyName': 'РиалВеб', 'EmailNotification': {'MoneyWarningVa

### Запрос трат за последние три дня по каждому аккаунту

In [8]:
budget_dict = yandex.get_account_spent(logins_actual)

Отчет для аккаунта lost-ark-mrt создан успешно
RequestId: 5877013937294907622
Отчет для аккаунта perfect-world-mrt создан успешно
RequestId: 4706172354498102420
Отчет для аккаунта arche-age-mrt создан успешно
RequestId: 5329712452103680422
Отчет для аккаунта allodsonline-games создан успешно
RequestId: 1019499461091834018
Отчет для аккаунта atomicheart-game создан успешно
RequestId: 1337553930295237864
Отчет для аккаунта battle-teams2 создан успешно
RequestId: 634219826079899205
Отчет для аккаунта warface2016-2017warfacerwyndxg создан успешно
RequestId: 2730839738598679817
Отчет для аккаунта revival-astrum создан успешно
RequestId: 2506643458563712885


### Преобразование временного файла из запроса в pandas df

In [9]:
budget_dict = pd.read_csv(StringIO(budget_dict), sep=',', escapechar='\\', index_col=False)

### Фильтруем аккаунты только со спендом за последние три дня

In [10]:
budget_dict = budget_dict[(budget_dict['Costs']!= 0)]

In [11]:
merged_df = pd.merge(budget_dict, df, how='left', on='Login').fillna(0)

In [12]:
merged_df['avg_cost'] = round(merged_df['Costs'] / 3, 2)
merged_df['days'] = pd.to_numeric(merged_df['Amount'] // merged_df['avg_cost'], downcast='integer')

## Отправка сообщений в телеграм

### Вариант с одним сообщением

In [13]:
tg_message = []

Дальше собираем список для сообщения

In [14]:
for row in merged_df.index:
    if merged_df['days'][row] == 0 and merged_df['Amount'][row] < 5000:
        tg_message.append(f"На аккаунте {merged_df['Login'][row]} закончился бюджет.")
    elif merged_df['days'][row] == 0 and merged_df['Amount'][row] > 5000:
        tg_message.append(f"На аккаунте {merged_df['Login'][row]} бюджета меньше чем на день.")
    elif merged_df['days'][row] == 1:
        tg_message.append(f"{merged_df['Login'][row]} хватит бюджета на {merged_df['days'][row]} день.")
    elif 1 < merged_df['days'][row] < 4:
        tg_message.append(f"{merged_df['Login'][row]} хватит бюджета на {merged_df['days'][row]} дня.")

Преобразуем список в строку с разделителями (каждая запись с новой строки + одна строка отступа)

In [15]:
tg_message = "\n\n".join(tg_message)

In [16]:
print(tg_message)

allodsonline-games хватит бюджета на 1 день.

atomicheart-game хватит бюджета на 2 дня.


In [17]:
# telegram_bot = TelegramBot(bot_token, chat_id)
# telegram_bot.send_message(tg_message)
if tg_message:
    tg_message = f"Бюджеты в Yandex: \n {tg_message}"
    ya_bot.send_text(tg_message)


### Алёрта об срабатывании скрипта

In [19]:
chat_id = '126841573'
telegram_bot = TelegramBot(bot_token, chat_id)
message_test = f"Yandex отработал"
telegram_bot.send_message(message_test)

{'ok': True,
 'result': {'message_id': 494,
  'from': {'id': 6077122053,
   'is_bot': True,
   'first_name': 'warface_guide_bot',
   'username': 'Warface_guide_bot'},
  'chat': {'id': 126841573,
   'first_name': 'Alexey',
   'last_name': 'Rusakov',
   'username': 'WhiteRabbits',
   'type': 'private'},
  'date': 1715066953,
  'text': 'Yandex отработал'}}